In [1]:
from scripts import setup_environment

setup_environment()

In [2]:
from scripts.dataset import BrainteaserDataset

dataset = BrainteaserDataset("data")

In [3]:
import string

from scripts.dataset import RiddleQuestion
from scripts.prompt_helpers import create_prompt_template


def args_generator(riddle_question: RiddleQuestion):
    template_args = {
        "question": riddle_question.question,
        "choices": "\n".join(
            [
                f"({string.ascii_uppercase[j]}) {choice}"
                for j, choice in enumerate(riddle_question.choice_list)
            ]
        ),
    }

    return template_args

In [4]:
from scripts.lmm import OllamaModelBuilder
from scripts.executor import Executor

base_url = "http://107.222.215.224:23251"
model_builder = OllamaModelBuilder(base_url)

executor = Executor(
    models=[
        # Llama3.1
        model_builder.build_model("llama3.1:8b-instruct-q8_0"),  # => 9 GB
        # Llama3.2
        model_builder.build_model("llama3.2:1b-instruct-fp16"),  # => 2.5 GB
        model_builder.build_model("llama3.2:3b-instruct-fp16"),  # => 6.4 GB
        # Phi3.5
        model_builder.build_model("phi3.5:3.8b-mini-instruct-fp16"),  # => 7.6 GB
        # Phi4
        model_builder.build_model("phi4:14b-q4_K_M"),  # => 9.1 GB
        # Qwen2.5
        model_builder.build_model("qwen2.5:0.5b-instruct-fp16"),  # => 1 GB
        model_builder.build_model("qwen2.5:1.5b-instruct-fp16"),  # => 3.1 GB
        model_builder.build_model("qwen2.5:3b-instruct-fp16"),  # => 6.2 GB
        model_builder.build_model("qwen2.5:7b-instruct-q8_0"),  # => 8.1 GB
        model_builder.build_model("qwen2.5:14b-instruct-q4_K_M"),  # => 9 GB
        model_builder.build_model("qwen2.5:32b-instruct-q4_K_M"),  # => 20 GB
        # Gemma2
        model_builder.build_model("gemma2:2b-instruct-fp16"),  # => 5.2 GB
        model_builder.build_model(
            "gemma2:9b-instruct-q8_0",
        ),  # => 9.8 GB
        model_builder.build_model("gemma2:27b-instruct-q4_K_M"),  # => 22 GB
        # Mistral Nemo
        model_builder.build_model("mistral-nemo:12b-instruct-2407-q4_K_M"),  # => 7.5 GB
    ]
)

2025-02-28 10:18:31,553 - INFO - Initialized executor with 15 models.


## Baseline Zero-Shot Evaluation

Testing performance with the minimal default system prompt: `You are an AI assistant.` without any task-specific instructions


In [5]:
from scripts.executor import Dataset

executor_data = [
    Dataset(name="sp", riddles=dataset.sp),
    Dataset(name="wp", riddles=dataset.wp),
]
chat_prompt_template = create_prompt_template("default")
results = await executor.aexecute(
    executor_data,
    chat_prompt_template,
    args_generator,
    dump_to_pickle=True,
    create_checkpoints=True,
    resume_from_checkpoint=True,
    run_name="baseline-zero-shot-evaluation",
)

2025-02-28 10:18:31,579 - INFO - Starting execution 'baseline-zero-shot-evaluation': 2 dataset(s) x 15 model(s) = 16785 riddle evaluations


baseline-zero-shot-evaluation:   0%|          | 0/16785 [00:00<?, ?it/s]

## Model-Specific System Prompt Optimization: Comparative Analysis of Zero-Shot Performance


In [6]:
import dill as pickle

# Get the best prompt type for each model
with open("results/best_system_prompts_by_model.pkl", "rb") as f:
    best_prompt_types = pickle.load(f)


def create_prompt_template_by_model(
    model_name: str,
    dataset_name: str,
):
    best_system_template_name = best_prompt_types[model_name][dataset_name][
        "prompt_type"
    ]
    return create_prompt_template(best_system_template_name)


def get_prompt_template(model_name: str, dataset: Dataset):
    # Split name after the b paramer, e.g., llama3.1:8b-instruct-fp16 => llama3.1:8b
    model_name = model_name[0 : model_name.index("b-") + 1]
    chat_prompt_template = create_prompt_template_by_model(model_name, dataset.name)
    return chat_prompt_template

In [7]:
results = await executor.aexecute(
    executor_data,
    get_prompt_template,
    args_generator,
    dump_to_pickle=True,
    create_checkpoints=True,
    resume_from_checkpoint=True,
    run_name="system_optimized_zero_shot_evaluation",
)

2025-02-28 10:18:32,246 - INFO - Starting execution 'system-optimized-zero-shot-evaluation': 2 dataset(s) x 15 model(s) = 16785 riddle evaluations


system-optimized-zero-shot-evaluation:   0%|          | 0/16785 [00:00<?, ?it/s]

2025-02-28 11:51:18,478 - INFO - Saving results to results/system-optimized-zero-shot-evaluation/system-optimized-zero-shot-evaluation_results.pkl
2025-02-28 11:51:23,555 - INFO - Execution 'system-optimized-zero-shot-evaluation' completed successfully.
